In [ ]:
import pandas as pd
import numpy as np
import re
from textblob import TextBlob, Word
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data = pd.read_csv("text_df.csv",index_col = 0)
data.head()
data.reset_index(drop=True,inplace=True)

In [ ]:
data.isna().sum()

texts     1
labels    0
dtype: int64

In [ ]:
for i, j in enumerate(data['labels']):
  try:
    j = int (j)
  except ValueError as e:
    print (f'error on line {i}')
    continue
data.drop(labels = [2477],inplace = True)
data.dropna(inplace = True)
data['labels'].astype(int);

error on line 2477


In [ ]:
stopwords_en = set(stopwords.words('english'))
def clean_text(data):
  clean_text = []
  for i in data['texts']:
    i = i.lower()
    i = TextBlob(i)
    i = " ".join([word.lemmatize() for word in i.words if word not in stopwords_en and not word.isdigit()])
    clean_text.append(i)
  return clean_text

In [ ]:
data['texts'] = clean_text(data)

In [ ]:
data.head()

,texts,labels
0,hey guy know breathing neck get bud anyway abl...,0
1,deed number mean nothing fair well puffy jacke...,0
2,nothing anyway give treat,0
3,child support,0
4,want massage,1


In [ ]:
import tensorflow as tf
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text

In [ ]:
x = data['texts'].values
y = data['labels'].values
X_train,X_test,y_train, y_test = train_test_split(x,y, test_size = 0.2, shuffle = True)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [ ]:
tokenizer = text.Tokenizer(num_words = 1000)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
maxlength = len(max(X_train, key = len))
X_train = sequence.pad_sequences(X_train,maxlen=maxlength)
X_test = sequence.pad_sequences(X_test,maxlen=maxlength)
index = tokenizer.word_index

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM,Bidirectional, GlobalMaxPool1D,Dropout
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

In [ ]:
model = Sequential()
model.add(Embedding(1000,64,input_length=X_train.shape[1]))
model.add(Bidirectional(LSTM(196,return_sequences=True,name='lstm_layer',dropout=0.1,recurrent_dropout=0.1)))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['acc'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 508, 64)           64000     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 508, 392)          409248    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 392)               0         
_________________________________________________________________
dropout (Dropout)            (None, 392)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 393       
Total params: 473,641
Trainable params: 473,641
Non-trainable params: 0
_________________________________________________________________


In [ ]:
chk_pt = ModelCheckpoint('keras_model',monitor='val_acc',save_best_only=True)
model.fit(X_train,y_train,batch_size=30,epochs=30,verbose=2,validation_split=0.2,callbacks=chk_pt)

Epoch 1/100


In [ ]:
predictions = model.predict(X_test)
predictions = np.round(predictions)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
score = accuracy_score(y_test,predictions)
cm = confusion_matrix(y_test,predictions)
print("score: {} cm: {}".format(score,cm))

score: 0.8264021365890881 cm: [[1135  277]
 [ 178 1031]]
